## One Class Classification

In [8]:
import sys
print(sys.path)

['/Users/karlevans/Documents/Data Science', '/opt/anaconda3/lib/python39.zip', '/opt/anaconda3/lib/python3.9', '/opt/anaconda3/lib/python3.9/lib-dynload', '', '/opt/anaconda3/lib/python3.9/site-packages', '/opt/anaconda3/lib/python3.9/site-packages/aeosa']


### 1. Data

### 2. Preprocessing

### 3. One Class SVM 

In [1]:
import optuna

/opt/anaconda3/envs/env-01/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# one-class svm for imbalanced binary classification
import pandas as pd

# from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
import optuna

In [ ]:
# Datasets
df=pd.read_csv('data-2.csv')
# generate dataset
#X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0, n_clusters_per_class=1, weights=[0.999], flip_y=0, random_state=4)


In [ ]:
df.shape
#df.head()
#df['is_fraud'].()

(8580255, 44)

In [ ]:
# Split Data Types for processing
ignore_features = []
cat_col = [col for col, type in df.dtypes.items() if type in ["category", 'object']and col not in ignore_features]
num_col = [col for col, type in df.dtypes.items() if type in ['float', 'int64']and col not in ignore_features]
features = num_col + cat_col

y=df['is_fraud']
X=df.drop(ignore_features, axis=1)

#df['target'] = np.where(df['target'] == "target", 1, 0)
#y= df['target'].values

In [ ]:
from typing import List
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn import linear_model

def build_pipeline(num_features: List[str], cat_features: List[str]) -> Pipeline:
    """Full pipeline

    This function constructs the whole pipeline for training

    Params:
        config (Dict): Config content from yaml
        num_features (List[str]): List of numeric features
        cat_features (List[str]): List of categorial feature

    Returns:
        Pipeline that contians pre-process, sampling (If specified) and model

    Note:
        * Config assumes we're in the `pipeline` level already
    """
    numeric_transformer = make_pipeline(SimpleImputer(strategy='mean'),
                                       StandardScaler())

    categorical_transformer = make_pipeline(SimpleImputer(strategy='most_frequent'),
        #SimpleImputer(strategy='constant', fill_value='missing'),
        OneHotEncoder(handle_unknown='ignore'#, min_frequency=0.05
                      ))

    preprocessor = make_column_transformer((numeric_transformer, num_features),
        (categorical_transformer, cat_features),
        remainder="passthrough")
    
    pipe = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', linear_model.LogisticRegression())
    ])
    return pipe

pipeline = build_pipeline(num_features=num_col, cat_features=cat_col)

In [ ]:

# Hyperparameter Tuning
from sklearn.model_selection import cross_val_score
import optuna.create_study
import numpy as np

X_train, X_test,y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=21)

def objective(trial):
    # Suggest hyperparameters
    params = {
        'model__penalty':'elasticnet',
        'model__dual': False,
        'model__solver':'saga',
        'model__tol': trial.suggest_int("tol", 0.000001,0.001),
        'model__C':trial.suggest_float("C", 0.0, 10.0),
        'model__max_iter':trial.suggest_int("max_iter", 10, 500, step=5),
        'model__l1_ratio': trial.suggest_float("l1_ratio", 0, 1.0, step=0.1)       
    }

    model = pipeline.set_params(**params)
    scores = cross_val_score(model, X_train, y_train, cv=10, scoring='f1', verbose=3, error_score='raise')
    
    return scores.mean()
    
# Create a study object
study = optuna.create_study(direction="maximize")

# Optimize the objective function
study.optimize(objective, n_trials=10)

print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)
print("Best value:", study.best_value)

best_params = study.best_params
prefix = "model__"
params = {prefix + k: v for k, v in best_params.items()} # Add prefix for set_params function

ModuleNotFoundError: No module named 'optuna'

In [ ]:
## View feature importance of holdout set model
## Evaluation holdout set

from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, ConfusionMatrixDisplay

best_model = pipeline.set_params(**params)

# Initialize and train the best model
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='binary')
recall = recall_score(y_test, y_pred, average='binary')
class_report = classification_report(y_test, y_pred)

# Print the evaluation metrics
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'Classification Report:\n{class_report}')

# Confusion Matrix
import matplotlib.pyplot as plt 
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)
plt.show()

In [ ]:
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.5, random_state=2, stratify=y)

In [ ]:

# define outlier detection model
model = OneClassSVM(gamma='scale', nu=0.01)
# fit on majority class
trainX = trainX[trainy==0]
model.fit(trainX)
# detect outliers in the test set
yhat = model.predict(testX)
# mark inliers 1, outliers -1
testy[testy == 1] = -1
testy[testy == 0] = 1
# calculate score
score = f1_score(testy, yhat, pos_label=-1)
print('F1 Score: %.3f' % score)

ValueError: could not convert string to float: '171-59-9000'

# Isolation Forest

In [ ]:
model = IsolationForest()